## Notebook Imports

In [1]:
import pandas as pd
import numpy as np

## Constants

In [2]:
TRAINING_DATA_FILE = 'train-data.csv'
TEST_DATA_FILE = 'test-data.csv'

VOCAB_SIZE = 2500

TOKEN_SPAM_PROB_FILE = 'prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'prob-ham.txt'
TOKEN_ALL_PROB_FILE = 'prob-all.txt'

TEST_FEATURE_MATRIX = 'test-features.txt'
TEST_TARGET_FILE = 'test-target.txt'

## Loading the preprocessed dataset

In [3]:
train_data = pd.read_csv(TRAINING_DATA_FILE).to_numpy()
test_data = pd.read_csv(TEST_DATA_FILE).to_numpy()

## Full matrix from a sparse matrix

In [4]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe. 
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens. 
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [5]:
%%time
full_train_data = make_full_matrix(train_data, VOCAB_SIZE)

CPU times: user 20.8 s, sys: 55.8 ms, total: 20.8 s
Wall time: 20.9 s


In [6]:
full_train_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,1,4,2,0,3,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,1,6,5,1,0,0,1,18,3,5,...,0,0,0,0,0,0,0,0,0,0
10,1,1,0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5790,0,0,2,0,1,1,0,0,2,2,...,0,0,0,0,0,0,0,0,0,0
5791,0,0,2,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5792,0,0,3,1,1,0,2,1,1,1,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model

In [7]:
prob_spam = full_train_data['CATEGORY'].sum() / len(full_train_data)
print('Probability of spam is:', prob_spam)

Probability of spam is: 0.3318170331058883


In [8]:
# Total number of words
x_train = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
email_lengths = x_train.sum(axis=1)
email_lengths

DOC_ID
3        16
4        27
6       135
8       633
10       92
       ... 
5790     46
5791     55
5792    107
5794     20
5796     41
Length: 3957, dtype: int64

In [9]:
n_of_words = email_lengths.sum()
n_of_words

341455

In [10]:
spam_lengths = email_lengths[full_train_data['CATEGORY'] == 1]
ham_lengths = email_lengths[full_train_data['CATEGORY'] == 0]

In [11]:
n_of_spam_words = spam_lengths.sum()
n_of_ham_words = ham_lengths.sum()

n_of_ham_words, n_of_spam_words

(186825, 154630)

In [12]:
# Summing the Tokens Occuring in Spam
train_spam_tokens = x_train.loc[full_train_data.CATEGORY == 1]
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1

# Summing the Tokens Occuring in Ham
train_ham_tokens = x_train.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [13]:
# P(Token | Spam) - Probability that a token occurs given the email is spam
prob_tokens_spam = summed_spam_tokens / (n_of_spam_words + VOCAB_SIZE)

# P(Token | Ham) - Probability that a token occurs given the email is nonspam
prob_tokens_ham = summed_ham_tokens / (n_of_ham_words + VOCAB_SIZE)

# P(Token) - Probability that token occurs
prob_tokens_all = x_train.sum(axis=0) / n_of_words

#### Saving the parameters

In [14]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

## Preparing the test data

In [15]:
%%time
full_test_data = make_full_matrix(test_data, VOCAB_SIZE)

CPU times: user 9.01 s, sys: 8.18 ms, total: 9.02 s
Wall time: 9.02 s


In [16]:
x_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data['CATEGORY']

In [17]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, x_test)